In [1]:
library(tidyverse)
library(repr)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [14]:
G20_economic_data <- read_csv("https://raw.githubusercontent.com/mparhar1/DSCI-100-Group-48-Project/main/WEOApri23G20.csv")
colnames(G20_economic_data) <- make.names(colnames(G20_economic_data), unique = TRUE)

head(G20_economic_data, n=3)

Rows: 836 Columns: 59
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (46): ISO, WEO Subject Code, Country, Subject Descriptor, Subject Notes,...
dbl  (2): WEO Country Code, Estimates Start After

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


WEO.Country.Code,ISO,WEO.Subject.Code,Country,Subject.Descriptor,Subject.Notes,Units,Scale,Country.Series.specific.Notes,X1980,⋯,X2020,X2021,X2022,X2023,X2024,X2025,X2026,X2027,X2028,Estimates.Start.After
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
213,ARG,NGDP_R,Argentina,"Gross domestic product, constant prices","Expressed in billions of national currency units; the base year is country-specific. Expenditure-based GDP is total final expenditures at purchasers' prices (including the f.o.b. value of exports of goods and services), less the f.o.b. value of imports of goods and services. [SNA 1993]",National currency,Billions,Source: National Statistics Office Latest actual data: 2021 National accounts manual used: System of National Accounts (SNA) 2008 GDP valuation: Market prices Reporting in calendar year: Yes Start/end months of reporting year: January/December Base year: 2004 Chain-weighted: No Primary domestic currency: Argentine peso Data last updated: 03/2023,357.389,⋯,624.295,689.211,725.346,726.797,741.333,756.161,771.287,786.714,802.448,2021
213,ARG,NGDP_RPCH,Argentina,"Gross domestic product, constant prices","Annual percentages of constant price GDP are year-on-year changes; the base year is country-specific. Expenditure-based GDP is total final expenditures at purchasers' prices (including the f.o.b. value of exports of goods and services), less the f.o.b. value of imports of goods and services. [SNA 1993]",Percent change,NA,"See notes for: Gross domestic product, constant prices (National currency).",0.7,⋯,-9.943,10.398,5.243,0.2,2,2,2,2,2,2021
213,ARG,NGDP,Argentina,"Gross domestic product, current prices","Expressed in billions of national currency units. Expenditure-based GDP is total final expenditures at purchasers' prices (including the f.o.b. value of exports of goods and services), less the f.o.b. value of imports of goods and services. [SNA 1993]",National currency,Billions,Source: National Statistics Office Latest actual data: 2021 National accounts manual used: System of National Accounts (SNA) 2008 GDP valuation: Market prices Reporting in calendar year: Yes Start/end months of reporting year: January/December Base year: 2004 Chain-weighted: No Primary domestic currency: Argentine peso Data last updated: 03/2023,--,⋯,27481.440,46282.070,"82,650.24","164,655.09","268,792.37","403,024.96","584,085.98","817,219.74","1,104,151.70",2021


In [12]:
tidy_G20_economic_data <- G20_economic_data |>
    select(Country, Subject.Descriptor,
           Units,
           X2010:X2023
          ) |>
    filter(Subject.Descriptor == "Gross domestic product per capita, current prices" & Units == "U.S. dollars" |
           Subject.Descriptor == "Volume of imports of goods and services" |
           Subject.Descriptor == "Volume of exports of goods and services" | 
           Subject.Descriptor == "Inflation, average consumer prices" & Units == "Percent change" |
           Subject.Descriptor == "Unemployment rate" 
          ) |>
    filter(Country != "Argentina" & Country != "Saudi Arabia") |>
    mutate(X2014 = as.numeric(gsub(",","", X2014)),
           X2015 = as.numeric(gsub(",","", X2015)),
           X2016 = as.numeric(gsub(",","", X2016)),
           X2022 = as.numeric(gsub(",","", X2022)),
           X2023 = as.numeric(gsub(",","", X2023)),
          ) |>
    pivot_longer(cols = X2010:X2023,
                 names_to = "Year",
                 values_to = "Metric"
                )

head(tidy_G20_economic_data)

Country,Subject.Descriptor,Units,Year,Metric
<chr>,<chr>,<chr>,<chr>,<dbl>
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2010,56579.03
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2011,67270.36
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2012,68450.31
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2013,65200.37
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2014,61615.05
Australia,"Gross domestic product per capita, current prices",U.S. dollars,X2015,51412.25
